In [ ]:
using Plots
#pyplot()

In [ ]:
parabol(x) = sum(u->u*u, x) 
# f(0,0) = 0, x_i ∈ [-10,10]

shvefel(x) = sum(u-> -u*sin(sqrt(abs(u))), x) 
# f(420.9687,420.9687) = -819?, x_i ∈ [-500,500]

rastrigin(x) = 10*length(x) + sum(u->u*u-10*cos(2*pi*u), x) 
# f(0,0) = 0, x_i ∈ [-5,5]

ekly(x) = -20exp(-0.2sqrt(0.5(x[1]*x[1]+x[2]*x[2]))) - exp(0.5(cospi(2x[1])+cospi(2x[2]))) + 20 + ℯ
# f(0,0) = 0, x_i ∈ [-5,5]

rosenbrok(x) = 100(x[2]-x[1]*x[1])^2 + (x[1]-1)^2 
# f(0,0) = 0, x_i ∈ [-5,5]

bill(x) = (1.5-x[1]+x[1]*x[2])^2 + (2.25-x[1]+x[1]*x[2]*x[2])^2 + (2.625-x[1]+x[1]*x[2]^3)^2 
# f(3,0.5) = 0, x_i ∈ [-5,5]

boot(x) = (x[1]+2x[2]-7)^2 + (2x[1]+x[2]-5)^2 
# f(1,3) = 0, x_i ∈ [-10,10]

bukin6(x) = 100sqrt(abs(x[2]-0.01x[1]*x[1])) + 0.01abs(x[1]+10) 
# f(-10,1) = 0, x_i ∈ [-15,-5; -3,3]

levy13(x) = sinpi(3x[1])^2 + (1+sinpi(3x[2])^2)*(x[1]-1)^2 + (1+sinpi(2x[2])^2)*(x[2]-1)^2 
# f(1,1) = 0, x_i ∈ [-10,10]

himmelblau(x) = (x[1]^2+x[2]-11)^2 + (x[1]+x[2]^2-7)^2 
# f(3,2)... = 0, x_i ∈ [-5,5]

camel3humped(x) = 2x[1]^2 - 1.05x[1]^4 + x[1]^6 /6 + x[1]*x[2] + x[2]^2 
# f(0,0) = 0, x_i ∈ [-5,5]

izom(x) = -cos(x[1])*cos(x[2])*exp(-( (x[1]-pi)^2 + (x[2]-pi)^2 )) 
# f(π,π) = -1, x_i ∈ [-100,100]

holdertable(x) = -abs(sin(x[1])*cos(x[2])exp(abs( 1-sqrt(x[1]^2+x[2]^2)/pi ))) 
# f(±8.05502,±9.66459) = -19.2085, x_i ∈ [-10,10]

shaffer4(x) = 0.5 + (cos(sin(abs(x[1]^2-x[2]^2)))^2-0.5) / (1+0.001(x[1]^2+x[2]^2))^2 
# f(0,1.25313) = 0.292579, x_i ∈ [-100,100]

In [ ]:
fun = ekly

In [ ]:
low = [-5 -5]
up = [5 5]
Xs = range(low[1], stop = up[1], length = 40)
Ys = range(low[2], stop = up[2], length = 40)
Zs = [ fun([x y]) for x in Xs, y in Ys ];
Zs = Zs'
contour(Xs, Ys, Zs)
#savefig("$fun surface.png")

In [ ]:
minimum(Zs)

In [ ]:
function ploter(l, u, xy, n_age, Fmin )
    contour(Xs, Ys, Zs, fill = true);
    scatter!(xy[1,:], xy[2,:], legend = false, xaxis=( (l[1], u[1])), yaxis=( (l[2], u[2])) );
    title!("age: $n_age, f(x) = $(round(Fmin, digits = 4))")
    savefig("plots//$fun $n_age.png")
    
end

https://habr.com/ru/post/440234/

In [ ]:
function mdpso(;
	nparts = 50,
	ndimes = 2,
	ages = 50, # количество эпох
	lover = [-10 -10],
	upper = [10 10],
	C1 = [1.9 1.9],
	C2 = [1.8 1.8],
	Ac = [0.1 0.1], # определяет длину шага
	)

    minind = 0;
    V = zeros(nparts,ndimes)
    X = zeros(nparts,ndimes)
    funmin = -Inf
    Fmin = Inf
    Fbest = fill(Fmin, nparts)
    funx = zeros(nparts)
    xmem = zeros(nparts,ndimes)
    xbest = zeros(ndimes) # лучшая координата

	# частицы разбрасываются по исследуемой области
	for i in 1:nparts, j in 1:ndimes
		X[i,j] = randomer(lover[j], upper[j])
	end
    
	for i in 1:ages
        
		for j in 1:nparts
			
			funx[j] = fun(X[j,:])
			if funx[j] < Fbest[j]
				Fbest[j] = funx[j]
				xmem[j,:] = X[j,:]
			end
		end
        
        ploter(lover, upper, X, funx, i);
        
		funmin = minimum(funx)
		minind = argmin(funx)

		if funmin < Fmin
			Fmin = funmin
			xbest[:] = X[minind,:]
		end

		for j in 1:nparts, k in 1:ndimes
			R1 = rand()
			R2 = rand()
			V[j,k] = Ac[k]*V[j,k] + C1[k]*R1*(xmem[j,k] - X[j,k]) + 
				C2[k]*R2*(xbest[k] - X[j,k])

			X[j,k] += V[j,k]
		end
		println("Age № $i\n xbest:\n $(xbest[1]) $(xbest[2])")
		println("Fmin: $Fmin\n")
	end
    
    f = open("$fun.txt","w")
    write(f,"C1 = $C1, C2 = $C2, Ac = $Ac, lower = $lover, upper = $upper, ages = $ages, parts = $nparts")
    close(f)
end

In [ ]:
mdpso(C1 = [0.7 0.7], C2 = [0.5 0.5], Ac = [0.09 0.09], lover = low, upper = up, ages = 10, nparts = 50);

* Зайцев В. В. Численные методы для физиков. Нелинейные уравнения и оптимизация: учебное пособие. – Самара, 2005г. – 86с.
* Иванов А. В. Компьютерные методы оптимизации оптических систем. Учебное пособие. –СПб: СПбГУ ИТМО, 2010 – 114с.
* Попова Т. М. Методы многомерной оптимизации: методические указания и задания к выполнению лабораторных работ по дисциплине «Методы оптимизации» для студентов направления «Прикладная математика»/ сост. Т. М. Попова. – Хабаровск: Изд-во Тихоокеан. гос. ун-та, 2012. – 44 с.

### Методы оптимизации

Решение задачи оптимизации заключается в поиске значений $x_1,...,x_n$ при которых функция $а(x_1,...,x_n)$ достигает своего минимума (максимума). На практике, задачи многомерной оптимизации, заключающиеся в поиске минимумов функций нескольких переменных, встречаются гораздо чаще, нежели задачи, сводящиеся к минимизации функции одного аргумента.

 Задачи оптимизации часто возникают в физических исследованиях и технических разработках. Например, минимум энергии физической системы определяет ее стационарное состояние. Поиск наиболее экономичной конструкции радиоэлектронного устройства проводится минимизацией энергопотребления, выраженного в виде функции параметров системы. Иногда оптимизационные задачи появляются опосредованно как средство решения каких-либо других задач. Так, краевую задачу для системы обыкновенных дифференциальных уравнений можно решать методом пристрелки, отыскивая нулевой минимум целевой функции, сконструированный из граничных условий. 
 
На подготовитетельном этапе на тестовых функциях были опробованы ряд методов, такие как:
* Метод покоординатного спуска, сводящий решение задачи многомерной оптимизации к многократному поиску минимума функци одного аргумента. В качестве методов одномерной оптимизиции были реализованы методы Ньютона, секущих, золотого сечения, а также методы последовательной и обратной параболической интерполяции
* Метод наискорейшего спуска - градиентный метод, использующий для поиска минимума не только значения целевой функции, но и значения ее градиента. Поиск экстремума ведется шагами в направлении градиента (max) или антиградиента (min). На каждом шаге в направлении градиента (антиградиента) движение осуществляется до тех пор, пока функция возрастает (убывает).
* Метод Нелдера — Мида, также известный как метод деформируемого многогранника и симплекс-метод, — метод безусловной оптимизации функции от нескольких переменных, не использующий производной(точнее — градиентов) функции, а поэтому легко применим к негладким и/или зашумлённым функциям. Суть метода заключается в последовательном перемещении и деформировании симплекса вокруг точки экстремума. Метод находит локальный экстремум и может «застрять» в одном из них. Если всё же требуется найти глобальный экстремум, можно пробовать выбирать другой начальный симплекс.



# Cat Swarm Optimization 

https://www.researchgate.net/publication/221419703_Cat_Swarm_Optimization

Согласно классификации биологии, семейство кошачьих содержит около тридцати двух различных видов существ, например, лев, тигр, леопард, кошка и т.д. Несмотря на то, что у них разные жизненные условия, у большинства кошачьих в поведении по-прежнему существует много общего. Для диких кошек охотничий навык обеспечивает выживание, но для домашних кошек он проявляется в виде естественного инстинкта сильного любопытства к любым движущимся вещам. Хотя они в большинстве своем проводят время неактивно.

У каждой кошки есть своя собственная позиция, состоящая из $M$ измерений, скорости для каждого измерения, значение пригодности, которое представляет приспособление кошки к оптимизируемой функции, и флага, чтобы определить, находится ли кошка в режиме поиска или в режиме отслеживания. 
Окончательное решение - лучшая позиция для одной из кошек, поскольку CSO сохраняет лучшее решение до конца итераций.

### Режим поиска

Эта подмодель используется для моделирования ситуации с кошкой, которая отдыхает, оглядывается вокруг и ищет следующую позицию для перехода. В режиме поиска мы определяем четыре существенных фактора: пул памяти поиска (SMP), диапазон поиска для выбранного измерения (SRD), количество направлений (CDC) и самоопределение (SPC). 

1. SMP используется для определения размера памяти поиска для каждой кошки, который указывает точки, которые приметила кошка. Кошка выберет точку из пула памяти в соответствии с правилами, описанными ниже. 
1. SRD объявляет мутативный коэффициент для выбранных измерений. В режиме поиска, если для изменения выбран размер разницы между новым значением и старым, то агент не выйдет за пределы диапазона, который определяется SRD. 
1. CDC раскрывает, сколько измерений будет варьироваться. Все эти факторы играют важную роль в режиме поиска. 
1. SPC - это логическая переменная, которая решает, будет ли точка, в которой кошка уже стоит, одним из кандидатов для перехода. Неважно, является ли значение SPC истинным или ложным; на значение SMP это не повлияет. 

Как работает режим поиска, можно описать в 5 этапов следующим образом: 

1. Сделать $j$ копии текущей позиции $cat_k$, где j = 1:SMP. Если значение SPC равно true, пусть j = (SMP-1), а затем сохранить текущую позицию в качестве одного из кандидатов. 
1. Для каждой копии, согласно CDC, случайным образом добавляется или уменьшается SRD в процентах от текущих значений и заменяются старые.
1. Рассчитываются значения пригодности ($f(l^k)$) всех направлений-кандидатов.
1. Если все $f(l^k)$ не совсем равны, вычислить вероятность выбора каждой точки-кандидата методом рулетки \cite{rulet}, в противном случае установить всю вероятность выбора каждой точки-кандидата равной 1.
$$
p^k=\frac{\displaystyle\left|\max_{j=1,\dots,N}f\left(l^j\right)-f\left(l^k\right)\right|}{\displaystyle\max_{j=1,\dots,N}f\left(l^j\right)-\min_{j=1,\dots,N}f\left(l^j\right)}.
$$
1. Из пула случайно выбрать точку, к которой нужно перейти и заменить текущую позицию для $cat_k$.

**Режим отслеживания** является подмоделью для моделирования случая кошки при отслеживании некоторых целей (другой кошки с лучшими координатами).
Как только кошка переходит в режим отслеживания, она движется в соответствии со своими собственными скоростями для каждого измерения. Действие режима трассировки можно описать в 3 этапа следующим образом:

1. Обновить скорости для каждого измерения ($v_{k, d}$) в соответствии с 
$$
v_{k,d} = v_{k,d} + r_1\times c_1 \times (x_{best,d} - x_{k,d}), \ d=1,2,...,M
$$

1. Проверить, находятся ли скорости в диапазоне максимальной скорости. Если новая скорость выходит за пределы допустимого диапазона, установить ее равной пределу.
1. Обновить положение $cat_k$ в соответствии с уравнением 
$$
x_{k,d} = x_{k,d} + v_{k,d}
$$

Как мы описали в вышеприведенном подразделе, CSO включает две подмодели: режим поиска и режим трассировки. Чтобы объединить два режима в алгоритм, мы определяем коэффициент смешивания (TS) режима поиска с режимом трассировки.

Поведение бега за целями кошки применяется в режиме трассировки. Следовательно, TS должно иметь достаточно большое значение, чтобы гарантировать, что кошки проводят большую часть времени в режиме поиска, как в реальном мире. 
Процесс CSO можно описать в 6 этапов следующим образом:

1. Создать N кошек. 
1. Случайным образом разогнать кошек в пространстве M-мерного решения и случайным образом выбрать значения, которые находятся в пределах максимальной скорости, для скоростей каждой кошки. Затем случайно выбрать количество кошек и перевести их в режим отслеживания согласно TS, а остальные переведите в режим поиска. 
1. Оценить пригодность каждой кошки, подставляя положения кошек в функцию пригодности, которая представляет критерии нашей цели, и сохранить координаты лучшей кошки в памяти. 
1. Переместить кошек в соответствии с их флагами. Если $cat_k$ находится в режиме поиска, инициировать алгоритм поиска, в противном случае применить процесс отслеживания. Этапы процесса представлены выше. 
1. Снова выбрать количество кошек и установить их в режим отслеживания в соответствии с TS, затем перевести других кошек в режим поиска. 
1. Проверить условие завершения, если оно выполнено, прекратить работу программы. В противном случае повторить шаги с 3 по 5.

Нетрудно понять, что метод оптимизации стаей кошек предоставляет более интеллектуальный и интерактивный поиск глобального минимума по сравнению с методом роя частиц. Лучшей стратегией поиска является исследование функционала и поиск глобального минимума с помощью роевого метода с последующим уточнением параметров с помощью одного из методов многомерной оптимизации.

https://habr.com/ru/post/328760/

Псевдокод алгоритма представлен ниже:

Шаг 1. Инициализировать популяцию из N котов.

Шаг 2. Определить приспособленность каждого кота на основе его позиции в исследуемом пространстве. Запомнить "лучшего" кота (в терминологии задачи минимизации, ему будет соответствовать наименьшее значение функции).

Шаг 3. Переместить котов в соответствии с их процедурой смещения (поиск или погоня).

Шаг 4. Заново присвоить котам режимы перемещения в соответствии с параметром MR.

Шаг 5. Проверить условие окончания работы. В случае его невыполнения перейти к шагу 2.

* количество котов (numberOfCats) — чем больше котов, тем дольше время работы алгоритма (если его ограничивать количеством итераций), но и потенциально большая точность найденного ответа,
* пропорция котов в режиме поиска и погони (MR) — данный параметр позволяет направлять поиск по той стратегии, которую пользователь считает предпочтительной; например, если вы заведомо знаете окрестность, в которой лежит глобальный оптимум, то логично инициализировать популяцию в этой окрестности и поддерживать большее число котов-исследователей в популяции для уточнения первоначального решения,
* количество попыток для режима поиска (SMP) — сколько разных смещений будет производить кот-исследователь; большие значения данного параметра увеличивают время одной итерации, но позволяют увеличить точность определения положения экстремума,
* доля смещения для режима поиска (SRD) — доля, на которую кот-исследователь смещается относительно своего текущего положения, большие значений смещают уточняющий поиск в сторону глобального,
* количество направлений, по которым ведется поиск (CDC) — данный параметр регулирует количество измерений, которые будут изменять у текущего положения кота, находящегося в режиме поиска; меньшие значения делают поиск покоординатным,
* желание остаться на старом месте (SPC) — булева переменная, которая позволяет выбирать, может ли кот-исследователь остаться не текущем месте,
* константа скорости (velocityConstant) — определяет степень поворотливости кота во время погони; большие значения быстрее меняют текущий вектор скорости кота,
* максимальная скорость (velocityRatio) — все-таки вы в доме хозяин, поэтому в случае если кто-то из котов слишком уж разогнался, то вы вполне можете на него прикрикнуть,чтобы он притормозил, т.о. данный параметр ограничивает максимальную скорость движения котов.


In [ ]:
function cso(fun;
    ncats = 20,
    ndimes = 2,
    ages = 20, # количество эпох
    TS = 0.5, # Seeking / Tracing
    SMP = 5, # Seek Memory Pool
    lover = [-10.; -10.],
    upper = [10.; 10.],
    dx = [ 0.1, 0.1], 
    a = 0.1 ) # acceleration

    randomer(a::Float64,b::Float64) = rand()*(b-a)+a
    minind = 0
    Fmin = Inf
    #xbest = zeros(ndimes) # лучшая координата
    P = zeros(SMP) # probabilities
    funx = zeros(SMP)
    dirs = zeros( ndimes, SMP )
    #Fbest = fill(Fmin, ncats)
    V = zeros(ndimes,ncats)
    X = zeros(ndimes,ncats)
    
    for i in 1:ndimes, j in 1:ncats
        X[i,j] = randomer(lover[i], upper[i])
    end
    
    Fbest = [ fun(X[:,j]) for j = 1:ncats ]

    Fmin, minind = findmin(Fbest)
    xbest = X[:,minind]


    for i in 1:ages
        
        for j in 1:ncats
            
            if rand() < TS #seeking
                for k in 1:SMP
                    dirs[:,k] = X[:,j] + rand()*dx
                    funx[k] = fun(dirs[:,k])
                end
                #
                minf, maxf = extrema( funx )
                
                P = [ abs(maxf - funx[k]) / (maxf - minf) for k = 1:SMP ]
                
                Ps = cumsum( [0.;P] )
                r = rand()
                loci = 1
                for k = 1:SMP
                    if Ps[k] < r < Ps[k+1]
                        loci = k
                        break
                    end
                end
                #
                #loci = argmin(funx) # так лучше?
                
                X[:,j] .= dirs[:,loci]
                if funx[loci] < Fbest[j]
                    Fbest[j] = funx[loci]
                end
                
                
            else # tracing
                V[:,j] .+= rand()*a*(xbest - X[:,j])
                X[:,j] .+= V[:,j]
                
                Ftr = fun( X[:,j] )
                if Ftr < Fbest[j]
                    Fbest[j] = Ftr
                end
                
            end
        end
        Fbox, minind = findmin(Fbest)
        if Fbox < Fmin
            Fmin = Fbox
            xbest .= X[:,minind]
        end 
    ploter(lover, upper, X, i, Fmin);
        println("Age № $i\n xbest:\n $xbest")
        println("Fmin: $Fmin\n")
    end
    
    #f = open("$fun.txt","w")
    #write(f,"C1 = $C1, C2 = $C2, Ac = $Ac, lower = $lover, upper = $upper, ages = $ages, parts = $nparts")
    #close(f)
end

In [ ]:
cso(ekly, ages = 20, lover = [-5., -5.], upper = [5., 5.], dx = [0.1, 0.1], a = 0.1)